# Выбор оптимальной модели

В этом семинаре мы потренируемся обучению, оценке и валидации моделей, подбору оптимальных гиперпараметров, смешиванию моделей. Вам предлагается решить задачу бинарной классификации, а именно построить алгоритм, определяющий превысит ли средний заработок человека порог $50k.

Импортируем необходимые библиотеки. Загрузим данные, уберём пропущенные значения

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
# Назначаем имена колонок
columns = ('age workclass fnlwgt education educ-num marital-status occupation relationship '
           'race sex capital-gain capital-loss  hours-per-week native-country salary')

numeric_indices = np.array([0, 2, 4, 10, 11, 12])
categorical_indices = np.array([1, 3, 5, 6, 7, 8, 9, 13])

df.columns = columns.split() #этот метод разделит датасет по колонкам как в массиве columns

df = df.replace('?', np.nan)

df = df.dropna()

df['salary'] = df['salary'].apply((lambda x: x==' >50K')) # Будем предсказывать 1(True), если зарплата больше 50K, 0(False) иначе

In [ ]:
numeric_data = df[df.columns[numeric_indices]]

categorial_data = df[df.columns[categorical_indices]]
dummy_features = pd.get_dummies(categorial_data)

In [ ]:
X = pd.concat([numeric_data, dummy_features], axis=1)
X.head()

In [ ]:
y = df['salary']

Теперь всё готово для обучения алгоритмов. Разбейте данные на train и test в соотношении 4:1.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = #ВАШ КОД

## Кросс-валидация

![img](https://lh6.googleusercontent.com/IgkIPN6nKjXJEHnbesY2bDCmGkJjSBqtkkF_yLRWlHX_hmfSQpfTC4tyPF13XQsqy9yrqMW4VIjyov-BjuzQKzf4yfFbrSO6HiMgPq9u_Lh5-h2Sdv9k1Mw5rJIDckKdJZ3IeppZpnI)

Вместо того, чтобы отделять валидационную выборку от тренировочной, проверяя построенные модели на ней, будем использовать метод кросс-валидации, реализованный вместе с поиском по сетке в классе sklearn.model_selection.GridSearchCV. Суть метода заключается в том, что мы разбиваем обучающую выборку на n частей (фолдов), обучаем алгоритм на каждых из n-1 фолдов и измеряем качество предсказания на оставшемся фолде, а затем усредняем результаты по всем фолдам.

Напишем функцию, визуализирующую поиск оптимального гиперпараметра модели по сетке. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def search_and_draw(X, y, model, param_name, grid, param_scale='ordinary', draw=True):
    parameters = {param_name: grid}
    
    CV_model = GridSearchCV(estimator=model, param_grid=parameters, cv=5, scoring='roc_auc', n_jobs=-1)
    CV_model.fit(X, y)
    
    means = CV_model.cv_results_['mean_test_score']
    error = CV_model.cv_results_['std_test_score']
    
    if draw:
        plt.figure(figsize=(15,8))
        plt.title('choose ' + param_name)


        if (param_scale == 'log'):
            plt.xscale('log')

        plt.plot(grid, means, label='mean values of score')

        plt.fill_between(grid, means - 2 * error, means + 2 * error, color='green', label='filled area between errors')
        plt.legend()
        plt.xlabel('parameter')
        plt.ylabel('roc_auc')
        plt.show()
        
    return means, error

Для моделей K ближайших соседей и решающего дерева найдите оптимальные параметры n_neighbors и max_depth. Для алгоритма KNN достаточно ограничиться числом соседей не более 100 и размером сетки не более 20. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
models = [KNeighborsClassifier(), DecisionTreeClassifier()]
param_names = ['n_neighbors', 'max_depth']
grids = #ВАШ КОД: создайте сетки для перебора
param_scales = ['ordinary', 'ordinary']

In [ ]:
for model, param_name, grid, param_scale in zip(models, 
                                                param_names, 
                                                grids, 
                                                param_scales):
    search_and_draw(X_train, y_train, model, param_name, grid, param_scale)

Подберём параметр n_estimators в алгоритме случайный лес. Известно, что случайный лес не переобучается. Поэтому график качества будет монотонно возрастать. Следовательно, необходимо найти минимальное значение n_estimators, при котором качество не изменяется. 
Поскольку каждое дерево обучается независимо от остальных, достаточно обучить сразу лес из большого количества деревьев, а затем рассмотреть подмножества нужного размера из исходного множества деревьев.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [ ]:
max_trees = 200

values = np.arange(max_trees) + 1

kf = KFold(n_splits=5, shuffle=True, random_state=1234)

global_scores = []

for train_indices, val_indices in kf.split(X_train):
    scores = []
    
    X_train_kf = #ВАШ КОД
    y_train_kf = #ВАШ КОД
    
    X_val_kf = #ВАШ КОД
    y_val_kf = #ВАШ КОД
    
    forest = RandomForestClassifier(n_estimators=max_trees)
    
    #ВАШ КОД: обучите лес на n-1 фолде
    
    trees = forest.estimators_
    
    for number_of_trees in values:
        thinned_forest = #ВАШ КОД: определите лес из первых number_of_trees деревьев обученного леса

        scores.append(roc_auc_score(y_val_kf, thinned_forest.predict_proba(X_val_kf)[:, 1]))
    
    scores = np.array(scores)
    
    global_scores.append(scores)

global_scores = np.stack(global_scores, axis=0)

In [ ]:
mean_cross_val_score = global_scores.mean(axis=0)
std_cross_val_score = global_scores.std(axis=0)

plt.figure(figsize=(15,8))
plt.title('Quality of random forest')

plt.plot(values, mean_cross_val_score, label='mean values')
plt.fill_between(values, 
                 mean_cross_val_score - 2 * std_cross_val_score, 
                 mean_cross_val_score + 2 * std_cross_val_score, 
                 color='green', 
                 label='filled area between errors')
plt.legend()
plt.xlabel('number of trees')
plt.ylabel('roc-auc')

plt.show()

Нормируем признаки и проделаем тот же эксперимент с алгоритмом ближайших соседей. Посмотрим, изменилось ли качество предсказания. В этот раз будем обучаться только на численных признаках: иначе KNeighborsClassifier будет учиться вечность.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#ВАШ КОД: преобразуйте выборки X_train и X_test

In [ ]:
models = [KNeighborsClassifier(), DecisionTreeClassifier()]
param_names = ['n_neighbors', 'max_depth']
grids = #ВАШ КОД
param_scales = ['ordinary', 'ordinary']

for model, param_name, grid, param_scale in zip(models, 
                                                param_names, 
                                                grids, 
                                                param_scales):
    search_and_draw(X_train[:, numeric_indices], y_train, model, param_name, grid, param_scale)

Проведём эксперимент над алгоритмом ближайших соседей для оптимального значения глубины из предыдущего пункта на всех признаках.

In [ ]:
# ВАШ КОД

Сделаем выводы. Какой из алгоритмов сработал лучше всего? Выберем его и вычислим итоговое качество на test.

In [ ]:
model = #ВАШ КОД: ваша лучшая модель

#ВАШ КОД

y_predicted = #ВАШ КОД: предскажите вероятности на test

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
auc = roc_auc_score(y_test, y_predicted)

plt.figure(figsize=(10,7))
plt.plot(*roc_curve(y_test, y_predicted)[:2], label='test AUC=%.4f' % auc)
plt.legend(fontsize='large')
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100))
plt.grid()
plt.show()

## Что ещё можно делать:

Мы подбирали оптимальный одномерный параметр для алгоритма. Можно также:

* Искать по сетке не только численные гиперпараметры, но и категориальные, например, метрику в алгоритме ближайших соседей или критерий ветвления в решающем дереве.

* Искать оптимальный параметр по многомерной сетке. Перебрать все возможные варианты здесь не выйдет, потому что на это уйдёт слишком много времени. Зато можно перебирать случайные точки по сетке. Эта процедура называется Grid Random Search.

# Стекинг

Идея стекинга состоит в том, чтобы обучать разнообразные алгоритмы и использовать их в качестве новых признаков объектов. 

Чтобы избежать переобучения, необходимо разделить обучающую выборку на n фолдов. Для предсказания ответов на k-ом фолде алгоритм обучается на оставшихся n-1 фолдах и предсказывает ответ на k-ом фолде. Такую схему обучения-предсказания реализует функция sklearn.model_selection.cross_val_predict.

In [ ]:
from sklearn.model_selection import cross_val_predict

Будем работать с тем же самым датасетом, что и ранее. Посмотрим, сумеем ли мы побить результаты случайного леса с помощью стекинга.

In [ ]:
def compute_meta_feature(model, X_train, X_test, y_train, cv):
    try:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
        model.fit(X_train, y_train)
        return train_answers, model.predict_proba(X_test)[:, 1]
    
    except Exception:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict')[:, 1]
        model.fit(X_train, y_train)
        return train_answers, model.predict(X_test)[:, 1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
models = #Список ваших любимых моделей из предыдущих пунктов

In [ ]:
meta_features_train = np.zeros((X_train.shape[0], 0))
meta_features_test = np.zeros((X_test.shape[0], 0))

In [ ]:
for model in models:
    train, test = compute_meta_feature(model, X_train, X_test, y_train, 5)
    meta_features_train = np.append(meta_features_train, train.reshape((train.size, 1)), axis=1)
    meta_features_test = np.append(meta_features_test, test.reshape((test.size, 1)), axis=1)

Выберите итоговую модель, которая будет обучаться на метапризнаках. Обучите её и сравните качество на test.

In [ ]:
stacking_model = #ВАШ КОД: заведите модель

#ВАШ КОД: обучите модель

y_predicted = #ВАШ КОД

In [ ]:
auc = roc_auc_score(y_test, y_predicted)

plt.figure(figsize=(10,7))
plt.plot(*roc_curve(y_test, y_predicted)[:2], label='test AUC=%.4f' % auc)
plt.legend(fontsize='large')
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100))
plt.grid()
plt.show()

# Бустинг

Попробуем в пару-тройку строк побить всё то качество, которого мы так усердно добивались.

In [ ]:
import xgboost

In [ ]:
boosting_model = xgboost.XGBClassifier(n_estimators=500)

boosting_model.fit(X_train, y_train)

y_predicted = boosting_model.predict_proba(X_test)[:, 1]

In [ ]:
auc = roc_auc_score(y_test, y_predicted)

plt.figure(figsize=(10,7))
plt.plot(*roc_curve(y_test, y_predicted)[:2], label='test AUC=%.4f' % auc)
plt.legend(fontsize='large')
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100))
plt.grid()
plt.show()